In [107]:
import numpy as np
import pandas as pd
import folium
from haversine import haversine

### 만들어 둔 CSV 파일 불러오기

In [108]:
total_df = pd.read_csv("total.csv", encoding = 'ms949')
total_df.drop('Unnamed: 0', axis=1, inplace=True)

load_df = pd.read_csv("total_load_points.csv", encoding = 'ms949')
load_df.drop('Unnamed: 0', axis=1, inplace=True)
load_df.head(5)

total_df.head(5)
print(len(total_df)) #2090


2090


### 중복데이터 제거

In [113]:

#a = total_df.groupby('명칭')['명칭'].count().sort_values(ascending = False)
#b = total_df.join(a, on='명칭', rsuffix='_r')

total_df = total_df.drop_duplicates(keep = 'first')
total_df.reset_index(drop=True, inplace = True)
print(len(total_df)) #1722
total_df.head(4)

1722


,분류,명칭,위도,경도
0,보안등,역삼1동833,37.493349,127.032033
1,보안등,역삼1동832-25,37.492834,127.031413
2,보안등,역삼1동832-27,37.492632,127.031508
3,보안등,역삼1동832-28,37.492480,127.031581


### 모든 데이터들을 담을 데이터 프레임을 만든다.

밝기 = 각 시설물의 점수      EX) 파출소 = 5점, 보안등 = 4점, 편의점 = 3점, cctv = 2점
총밝기 = '기준 시설물의 점수' + '주변 시설물의 점수'

In [114]:
all_df = pd.DataFrame(columns=['feature_ID', '밝기', '분류', '명칭', '위도', '경도'])

### 시설물에 밝기 점수 주기
for i in range(len(total_df)) :
    score = 0 
    if total_df['분류'][i] == '파출소' :
        score += 50
    elif total_df['분류'][i] == '보안등' :    
        score += 20
    elif total_df['분류'][i] == '편의점' :    
        score += 15
    else :    
        score += 15
        
    tmp = {'feature_ID' : 'Point_%d' % i, 
           '밝기' : score, 
           '분류' :total_df.loc[i,'분류'], '명칭' : total_df.loc[i,'명칭'], 
           '위도' : total_df.loc[i,'위도'], '경도' : total_df.loc[i,'경도'] 
           } 
    all_df = all_df.append(all_df.from_dict(tmp, orient = 'index').T)

all_df.reset_index(inplace=True,drop=True)
all_df.head(3)

,feature_ID,밝기,분류,명칭,위도,경도
0,Point_0,20,보안등,역삼1동833,37.4933,127.032
1,Point_1,20,보안등,역삼1동832-25,37.4928,127.031
2,Point_2,20,보안등,역삼1동832-27,37.4926,127.032


In [116]:
#엄청느림
# 총밝기 구하기    
scores = []
for i in range(len(load_df)) :
    score = 0
    s_point = (load_df.loc[i,'위도'], load_df.loc[i,'경도']) # 시작점
    for n in range(len(all_df)) :
        t_point = (all_df.loc[n,'위도'], all_df.loc[n,'경도']) # 타겟
        d_m = haversine(s_point, t_point, unit='m') #  시작점과 타겟의 거리  단위 미터
        # 반경 50M 를 범위로 잡음
        if d_m <= 50 :
            score += all_df.loc[n, '밝기']
    # print(i, score) # 잘나옴
    scores.append(score)

In [122]:
load_df['밝기'] = scores
# DataFrame 을 csv 파일로 저장하기
load_df.to_csv("load_df_with_light.csv", mode='w', encoding='ms949')   # encoding 을 안 하면 한글이 깨짐

In [120]:
all_df

,feature_ID,밝기,분류,명칭,위도,경도
0,Point_0,20,보안등,역삼1동833,37.4933,127.032
1,Point_1,20,보안등,역삼1동832-25,37.4928,127.031
2,Point_2,20,보안등,역삼1동832-27,37.4926,127.032
3,Point_3,20,보안등,역삼1동832-28,37.4925,127.032
4,Point_4,20,보안등,역삼1동832-29,37.4923,127.032
5,Point_5,20,보안등,역삼1동832-31,37.492,127.032
6,Point_6,20,보안등,역삼1동837-33,37.4916,127.032
7,Point_7,20,보안등,역삼1동837-15,37.4919,127.032
8,Point_8,20,보안등,역삼1동837-16,37.492,127.032
9,Point_9,20,보안등,역삼1동836,37.492,127.033
